# Airtravel Optimization Program

The below code is a our implementation of a program that calculates the lowest cost route for a aircraft to take. The user provides a csv file, that is read, line by line (containing the airports needed to go and aircrft used) and determines the optimal route to take in order to minimise the costs. 

The base currency is euro. In order to refuel the aircraft, you must purchase the refuel the aircraft to full using the local currecy. One kilometer requires one unit of fuel and can be purchased with one unit of the local currency. This means that the gain and loss of the cost of fuel is in the conversion rate between euro and the local currency.

To complete this exercise, we have implemented various data structres and algorithms. These will be explained as we make our way down the notebook.

In [1]:
import csv
import os
from math import pi, sin, cos, acos, floor

## Preparing

Below we have developed both functions and classes that are used at the beginning of the program. The following is a breakdown:

1. **Read file function:** The function reads in a csv file and creates a list for each line in the csv. Each line in the csv is then appended to a bigger datamatrix list. This list will contain the airports that are to be visited and the aircraft to be used.
2. **Aircraft class:** This class builds the aircraft to be used. Each aircraft object is to be assigned the model, maundacturer, max capacity and the currency capacity of the aircraft. Approprite setters and getters are implemented in order to access the object variables later in the program.
3. **Airport class**: Similar to the aircraft, each airport that is needed in the program is instantiated and is variables, airport code, longitutde, latitude and the exchange rate from euro is passed.

In [2]:
def read_file(file_name):
    """Read data in the file and Create Matrix"""
    datamatrix = []
    datafile = open(file_name)
    for line in datafile:
        datamatrix.append(line.split(","))
    datafile.close()
    return datamatrix

In [3]:
def buildAircraft():
    """Builds objects for each of the aircraft - with attributes model, manufacturer, and range.
    Returns a dictionary of this"""
    aircraftDict = {}
    with open('aircraft.csv', newline='', encoding="utf8") as airplane_file:  # opens the csv file
        reader = csv.reader(airplane_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for airplane in reader:  # iterates through the reader
            if airplane[2] == "imperial":
                airRange = int(airplane[4]) * 1.609
            else:
                airRange = airplane[4]
            aircraftDict[airplane[0]] = Aircraft(airplane[0], airplane[3], airRange)
    return aircraftDict

In [4]:
class Aircraft: # Class that contains all aircraft
    
    def __init__(self, model, manufacturer, max_capacity, current_capacity = 0):
        self._model = model # model of the aircraft
        self._manufacturer = manufacturer # manufacturer of the aircraft
        self._max_capacity = max_capacity # max capacity of the aircraft
        self._current_capacity = current_capacity # current capacity of the aircraft
    
    @property
    def max_capacity(self):
        return self._max_capacity # returns the max distance
    @property
    def manufacturer(self):
        return self._manufacturer # returns the manufacturer
    @property
    def model(self):
        return self._model # returns the model
    @property
    def current_capacity(self):
        return self._current_capacity # returns the current capacity left in the plane

class Airport:
    
    def __init__(self, airport_code, longitude, latitude, exchangeRate):
        self.airport_code = airport_code # airport code of the airport
        self._longitude = longitude # longitude of the airport
        self._latitude = latitude # latitude of the airport
        self._exchange_rate = exchangeRate # currency used by the airport
        
    @property
    def longitude(self):
        return self._longitude # returns the longitude of the airport
    @property
    def latitude(self):
        return self._latitude # returns the latitude of the airport
    @property
    def exchange_rate(self):
        return self._exchange_rate # returns the currency used by the airport
    @exchange_rate.setter
    def exchange_rate(self, exchangeRate):
        self._exchange_rate = exchangeRate # sets the currency of the airport
        

## Building the Aircraft

Time complexity: O(n)

Below is a function that builds each of the aircraft.

In [5]:
def buildAircraft():
    """Builds objects for each of the aircraft - with attributes model, manufacturer, and range.
    Returns a dictionary of this"""
    aircraftDict = {}
    with open('aircraft.csv', newline='', encoding="utf8") as airplane_file:  # opens the csv file
        reader = csv.reader(airplane_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for airplane in reader:  # iterates through the reader
            if airplane[2] == "imperial":
                airRange = int(airplane[4]) * 1.609
            else:
                airRange = airplane[4]
            aircraftDict[airplane[0]] = Aircraft(airplane[0], airplane[3], airRange)
    return aircraftDict

## Building Airport Objects

Time complexity: 2n^2 + 4n ----> O(n^2)

In [38]:
def buildAirports(listy):
    """Takes in a list of the route to be analysed. The airports are searched for in the csv.
    Objects of that airport are then created - with attributes latitude, longitude, and the exchange rate to Euro.
    It returns a dictionary with airport_code as key and object as value"""
    listx = listy[:-1]
    airport_list = []  # creates a new list

    with open('airport.csv', newline='', encoding="utf8") as airport_file:  # opens the csv file
        reader = csv.reader(airport_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for airport in reader:  # iterates through the reader
            if airport[4] in listx:
                airport_code = airport[4]  # assigns variable
                country_name = airport[3]  # assigns variable
                longitude = airport[6]  # assigns variable
                latitude = airport[7]  # assigns variable
                templist = [airport_code, country_name, longitude, latitude]
                airport_list.append(templist)
    if (len(airport_list)) != 5:
        return False

    country_currency_list = []  # creates a new list

    with open('countrycurrency.csv', newline='', encoding="utf8") as countrycurrency_file:  # opens the csv file
        reader = csv.reader(countrycurrency_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for country in reader:  # iterates through the reader
            temp_list = [country[0], country[14]]  # temp list created
            country_currency_list.append(temp_list)  # appends temp list to the main list

    currency_list = []  # creates a new list

    with open('currencyrates.csv', newline='', encoding="utf8") as currencyrates_file:  # opens the csv file
        reader = csv.reader(currencyrates_file)  # reads the cotents to a variable
        next(reader, None)  # returns none at the end of the file
        for currency in reader:  # iterates through the reader
            temp_list = [currency[1],currency[2]]  # temp list created
            currency_list.append(temp_list)  # appends temp list to the main list

    #Outer for loops goes through list of countries and the currency they have. Inner loop will go through currency 
    #and exchange rate list, matches currency to exchange rate, and creates a final list of lists with the 
    #country and currency rate in each inner list.
    final_list = []
    for i in country_currency_list:
        for x in currency_list:
            if i[1] == x[0]:
                templist = [i[0], x[1]]
                final_list.append(templist)

    #Outer for loop will go through list of lists that contains airport, country, latitude, longitude, 
    #and the inner loop will go through the list of airports and currency information and then match 
    #them with the airport in the outer list. The inner loop will then extend 
    x = 0
    i = 0 
    while x < len(airport_list):
        while i < len(final_list):
            if airport_list[x][1] == final_list[i][0]:
                airport_list[x].extend(final_list[i])
                break
            i+=1
        x+=1
    # Make a dictionary with the Airport code as the key and the value being the airport object of that key
    finalAirports = {}
    for i in airport_list:
        finalAirports[i[0]] = Airport(i[0], i[2], i[3], i[5])

    return finalAirports



## Check Plane Capability

Should go here!!

## Distance

### Calculating all Route Permutations

Time complexity: O(n^2)

In [20]:
def permutation(lst): 
    if len(lst) == 0: 
        return [] 
    if len(lst) == 1: 
        return [lst] 
    l = [] 
    for i in range(len(lst)): 
        m = lst[i] 
        remLst = lst[:i] + lst[i+1:] 
        for p in permutation(remLst): 
            l.append([m] + p) 
    return l 

Time complexity: O(n)

In [22]:
def allPerms(listx):
    """ Creates permutations of all possible routes using the input list of airports and plane """
    
    a = listx[0] # takes the departure airport
    permutation_list = listx[1:-1] # creates a list that removes the departure airport and the aircraft

    count = 0
    permlist = []
    newpermlist = []
    perm = permutation(permutation_list) 
    for i in list(perm): 
        permlist.append(i)

    for perms in permlist:
        perms = [a] + list(perms) + [a]
        newpermlist.append(perms)

    return newpermlist


### Calculates distances between each airport pair

Time complexity: n^4

In [41]:
def distanceBetweenAirports(latitude1, longitude1, latitude2, longitude2):
    radius_earth = 6371  # km
    theta1 = longitude1 * (2 * pi) / 360
    theta2 = longitude2 * (2 * pi) / 360
    phi1 = (90 - latitude1) * (2 * pi) / 360
    phi2 = (90 - latitude2) * (2 * pi) / 360
    distance = acos(sin(phi1) * sin(phi2) * cos(abs(theta1 - theta2)) + cos(phi1) * cos(phi2)) * radius_earth
    return floor(distance)

In [42]:
def leg_distance_calculator(listx, airport_list):
    """Takes list of route and aircraft required. Also takes in dictionary of airport objects. Creates permutation
    list of airports. Using the objects passed, it finds the distance between each airport and saves the leg
    and distance as a key and value in a dictionary"""
    
    a = listx[0] # takes the departure airport
    permutation_list = listx[1:-1] # creates a list that removes the departure airport and the aircraft

    count = 0
    permlist = []
    perm = permutation(permutation_list) 
    for i in list(perm): 
        permlist.append(i)

    airport_distances = {}

    for perms in permlist:
        perms = [a] + list(perms) + [a]
        for i in range(0, len(perms) - 1):
            for j in airport_list:
                if j == perms[i]:
                    airport1 = airport_list[perms[i]]
                    for k in airport_list:
                        if k == perms[i + 1]:
                            airport2 = airport_list[perms[i + 1]]
                    distance = distanceBetweenAirports(float(airport1.longitude), float(airport1.latitude),
                                                       float(airport2.longitude), float(airport2.latitude))
                    if distance != 0:
                        airport_distances['_'.join([airport1.airport_code, airport2.airport_code])] = distance

    return airport_distances


## Costing

### Calculating the Cost of each leg

Time complexity: O(n^2)

In [43]:
def findLegCosts(leg_distance_dict, airport_object_dict):
    """Takes in leg dictionary and airport objects. For each leg, it takes the departure airport
    and gets the local currency conversion rate. It then multiplies it by the distance to get the cost of each leg.
    Returns a dictionary with the cost of each leg."""
    costDict = {}
    for i in leg_distance_dict:
        myKey = i[:3]
        x = 0
        for j in airport_object_dict:
            if myKey == j:
                cost = round(float(airport_object_dict[j].exchange_rate) * float(leg_distance_dict[i]), 2)
                costDict[i] = cost
            x += 1
    return costDict

### Calculating the Cost of each Rotue

Time complexity: O(n^2)

In [44]:
def findRouteCost(myList, costDict):
    """This returns the total cost of each route - using the cost of each leg. Returned in a dictionary
    with: Key: route(tuple) & Values: Total cost"""
    routeCostDict = {}
    x = 0
    while x < len(myList):
        i = 0
        cost = 0
        while i < (len(myList[x]) - 1):
            myKey = str(myList[x][i]) + "_" + str(myList[x][i + 1])
            cost += costDict[myKey]
            cost = round(cost, 2)
            i += 1
        routeCostDict[tuple(myList[x])] = cost
        x += 1
    return routeCostDict


## Check Aircraft Capability

Time complexity: O(n^2 + n)

In [45]:
def checkAircraftAllowed(dictAirplane, distanceDict, input_list):
    """Checks that the aircraft being used can do the route. Returns the routes that are
    only possible with the aircraft"""
    planeToFly = input_list[5]
    planeRange = dictAirplane[planeToFly].max_capacity
    distanceDict_copy = distanceDict.copy()
    for j in distanceDict_copy:
        if distanceDict_copy[j] < int(planeRange):
            distanceDict.pop(j)
    finalRouteDict_copy = finalRouteDict.copy()
    for i in finalRouteDict_copy:
        toRemove = False
        for j in distanceDict:
            x = 0
            while x < len(i) - 1:
                if str(i[x] + "_" + i[x + 1]) == j:
                    toRemove = True
                x += 1
        if toRemove:
            finalRouteDict.pop(i)

## Caching

In [46]:
def cacheRoutes(airportsInRoute,plane,cost,dictCache):
    ''''''
    firstAirport = airportsInRoute[0]
    middleAirports = airportsInRoute[1:-1]
    middleAirports.append(plane)
    middleAirports.append(str(cost))
    dictCache[firstAirport] = middleAirports

In [47]:
def checkCache(airportList, myCache):
    ''''''
    if len(myCache) != 0:
        for i in myCache:
            if airportList[0] == i:
                if set(airportList[1:-1]) == set(myCache[i][0:-2]) and airportList[-1].rstrip() == myCache[i][-2]:
                    #myString = "The cheapest route is" + str(i) + str(cacheDict[i]) + str(i) + "and its cost is :€" + str(cacheDict[i][5])) 
                    myReturn = [str(i)]
                    myReturn.extend(myCache[i][0:4])
                    myReturn.append(str(i))
                    myReturn.append(str(myCache[i][5]))
                    return myReturn
                else:
                    return False
    return False

## Analysis

In [48]:
dictOfAircrafts = buildAircraft()  # Creates aircraft objects
cacheDict = {}
print("Please enter the name of the file containing the routes: ")
file_name = input("> ")
while not os.path.isfile(file_name):
    print("I am sorry. I do not believe that file exists. Remember, it is case sensitive. Please try again.")
    print("=" * 75)
    print("")
    file_name = input("> ")

print(f"Opening {file_name}....")
datamatrix = read_file(file_name)
i = 0
while i < len(datamatrix):
    inputList = datamatrix[i]  # sample list passed
    cacheCheck = checkCache(inputList,cacheDict)
    if cacheCheck != False:
        print("The cheapest route is " + str(cacheCheck[0:-1]) + " and its cost is: €" + str(cacheCheck[-1]))
        i += 1
        continue
    else:
        pass
    inputList[-1] = inputList[-1].rstrip()
    airport_objects_dict = buildAirports(inputList)  # creates the objects for each airport
    if not airport_objects_dict:
        print("One of the airports specified does not exist, skipping this route.")
        i += 1
        continue

    # create all the possible routes
    all_routes_list = allPerms(inputList)

    # Finds distances and costs of each leg
    dict_routes_distances = leg_distance_calculator(inputList, airport_objects_dict)
    leg_costs = findLegCosts(dict_routes_distances, airport_objects_dict)

    # Finds total route cost
    finalRouteDict = findRouteCost(all_routes_list, leg_costs)

    # Removes distances that the aircraft cannot do
    checkAircraftAllowed(dictOfAircrafts, dict_routes_distances, inputList)
    if (len(finalRouteDict)) == 0:
        print("This route is not possible with the " + str(inputList[-1]) + ". Try another plane. ")
        i += 1
        continue
    cheapestRoute = min(finalRouteDict, key=finalRouteDict.get)
    cost = finalRouteDict[cheapestRoute]
    cacheRoutes(list(cheapestRoute), inputList[-1].rstrip(), cost, cacheDict)
    print("The cheapest route is " + str(cheapestRoute) + " and its cost is: €" + str(cost))
    i += 1


Please enter the name of the file containing the routes: 
> test.csv
Opening test.csv....
This route is not possible with the 777. Try another plane. 
The cheapest route is ('SNN', 'ORK', 'CDG', 'SIN', 'MAN', 'SNN') and its cost is: €19773.1
The cheapest route is ('BOS', 'ORD', 'SFO', 'DFW', 'ATL', 'BOS') and its cost is: €8918.71
The cheapest route is ('DUB', 'LHR', 'MOS', 'HEL', 'CPH', 'DUB') and its cost is: €4434.29
The cheapest1 route is ['BOS', 'ORD', 'SFO', 'DFW', 'ATL', 'BOS'] and its cost is: €8918.71


### Unit tests

- Passing a plane that doesnt exist **DONE
- Passing a plane that can't fly **DONE
- Check to make sure that no two consecutive airports are the same **DONE
- airport that doesn't exist **DONE
- check that units match with assumptions
- Check that all exchange rates are there **DONE

## TODO

- Ask about unit tests
- Ask about O(n^4)

## Number of data structures/algorithms: 6
 - List
 - Tuple
 - dictionary 
 - Pricing algorithm
 - Plane allowed to fly route algorithm
 - Leg and distance calculation algorithm